# Capstone Project Notebook

In [1]:
import numpy as np 
import pandas as pd

In [2]:
print(" Hello Capstone Project Course!")

 Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

In [3]:
import pandas as pd
import numpy as np
import requests

In [4]:
from bs4 import BeautifulSoup

In [5]:
 # Loadin data from wikipedia
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [6]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [7]:
# create three lists to store table data
postalCodeList = []
boroughList = []
neighborhoodList = []

In [8]:
# loop through to find postcode, borough, neighborhood 
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))

In [9]:
toronto_df = pd.DataFrame({"PostalCode": postalCodeList,"Borough": boroughList,"Neighborhood": neighborhoodList})

toronto_df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [10]:
# Data Cleansing - Droping Borough which are not assigned and groupby neighbourhood in the same borough
toronto_df_dropna = toronto_df[toronto_df.Borough != "Not assigned"].reset_index(drop=True) # Droping borough cell
toronto_df_grouped = toronto_df_dropna.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x)) #groupby neighbourhood in the same borough
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]

In [11]:
toronto_df_grouped.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [12]:
column_names = ["PostalCode", "Borough", "Neighborhood"]
test_toronto_df = pd.DataFrame(columns=column_names)
code_list= ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for code in code_list:
    test_toronto_df = test_toronto_df.append(toronto_df_grouped[toronto_df_grouped["PostalCode"]==code], ignore_index=True)

test_toronto_df 

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Woodbine Gardens, Parkview Hill"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Maryvale, Wexford"
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."


In [13]:
# shape of the cleansed dataframe
toronto_df_grouped.shape

(103, 3)

In [54]:
# load the coordinates from the csv file and merging into the toronto_df_grouped
url="https://raw.githubusercontent.com/Ichigo-lab/Coursera_Capstone/master/Geospatial_Coordinates.csv"
coordinates =pd.read_csv(url)
coordinates.rename(columns={"Postal Code": "PostalCode"}, inplace=True)
toronto_df_coord = toronto_df_grouped.merge(coordinates, on="PostalCode", how="left")
toronto_df_coord.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [56]:
column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
test_toronto_df = pd.DataFrame(columns=column_names)
code_list= ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]
for code in code_list:
    test_toronto_df = test_toronto_df.append(toronto_df_coord[toronto_df_coord["PostalCode"]==code], ignore_index=True)

test_toronto_df 

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
1,M2H,North York,Hillcrest Village,43.803762,-79.363452
2,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
3,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
4,M4G,East York,Leaside,43.709060,-79.363452
5,M4M,East Toronto,Studio District,43.659526,-79.340923
6,M1R,Scarborough,"Maryvale, Wexford",43.750072,-79.295849
7,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437
8,M9L,North York,Humber Summit,43.756303,-79.565963
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
